In [1]:
import math
import random
import os
import time
import json
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

### Check Version

In [2]:
tf.__version__

'2.9.1'

### GPU Configuration

In [3]:
gpus = tf.config.list_physical_devices('GPU')

2022-11-09 21:25:24.530996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 21:25:24.537509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 21:25:24.537748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.experimental.set_memory_growth(gpus[0], True)

### Pandas Config

In [6]:
pd.set_option('display.max_column', None)

### Input Pipeline

In [7]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4

In [8]:
CROP_WIDTH = 227
CROP_HEIGHT = 227

In [9]:
foldFiles = ["adience/unprocessed/fold_0_data.txt",
             "adience/unprocessed/fold_1_data.txt",
             "adience/unprocessed/fold_2_data.txt",
             "adience/unprocessed/fold_3_data.txt",
             "adience/unprocessed/fold_4_data.txt"]

In [10]:
genderMap = defaultdict(lambda : np.NaN)
genderMap['m'] = 0
genderMap['f'] = 1

In [11]:
ages = ['(0, 2)', '(4, 6)', '(8, 13)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
ageMap = defaultdict(lambda : np.NaN)
for i,a in enumerate(ages):
    ageMap[a] = i

### Dataset Generation

Dataset generation pipeline:
Input: 
- foldFile - Path to fold file. Fold file Describes which images are in the fold and its corresponding labels
- imgBaseFolder - Base folder to search image from
- imgPrefix - Prefix of image file
- genderMap - Map from ['m', 'f', 'u', None], to one hot index
- ageMap - Map from age category to one hot index
- imgWidth - Resulting image width
- imgHeigh - Resulting image height
- batchSize - Int or None, batch size
- configureDs - Function accepting dataset for performance configurations
- preBatch - List of (name, functions) pair that will be mapped before batching. name is used as name parameters for tf graph 
- postBatch - List of (name, functions) that will be mapped after batching. name is used as name parameters for tf graph

The processing functions should have signature function(img, label) -> (img, label)

Pipeline

Read Fold File -> Preprocess filename and labels (dataframe) -> Convert filename and labels to numpy array -> Convert filename and labels to tf dataset -> Parse images and labels -> Configure Dataset for performance -> Pre-Batching preprocessing -> Batch -> Post-Batching preprocessing -> Output

Some preprocessing steps can only be done before and some can only be done after batching, thats why there are seperated pre and post batching list

In [12]:
def generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, batchSize, configureDs=None, preBatch=[], postBatch=[]):
    
    def parseImage(filename):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img)
        img = tf.image.resize(img, [imgHeight, imgWidth])
         
        return img
    
    # Read Fold File
    foldData = []
    for f in foldFiles:
        foldData.append(pd.read_csv(f, sep="\t"))
    foldData = pd.concat(foldData)
    
    # Form File Name
    foldData['filename'] = foldData.apply(lambda r: os.path.join(imgBaseFolder, r['user_id'], f"{imgPrefix}.{r['face_id']}.{r['original_image']}"), axis=1)
    
    # Generate Label One Hot Index
    foldData['gender_ind'] = foldData['gender'].map(genderMap)
    foldData['age_ind'] = foldData['age'].map(ageMap)
    
    # Remove dirty data
    foldData.dropna(subset=['gender_ind', 'age_ind'], inplace=True)
    
    # Dataframe to numpy
    filenames = foldData['filename'].to_numpy()
    
    genderIndex = foldData['gender_ind'].to_numpy().astype(int)
    ageIndex = foldData['age_ind'].to_numpy().astype(int)
    
    # Numpy to Dataset
    fnDs = tf.data.Dataset.from_tensor_slices(filenames)
    genderIndDs = tf.data.Dataset.from_tensor_slices(genderIndex)
    ageIndDs = tf.data.Dataset.from_tensor_slices(ageIndex)
    
    # Parse Images
    imageDs = fnDs.map(parseImage, num_parallel_calls=tf.data.AUTOTUNE, name="parse_image")
    
    # Parse Labels
    genderLabDs = genderIndDs.map(lambda x: tf.one_hot(x, genderDepth), num_parallel_calls=tf.data.AUTOTUNE, name="gender_one_hot")
    ageLabDs = ageIndDs.map(lambda x: tf.one_hot(x, ageDepth), num_parallel_calls=tf.data.AUTOTUNE, name="age_one_hot")
    
    # Combine Labels
    labelDs = tf.data.Dataset.zip((genderLabDs, ageLabDs), name="label_zip")
    labelDs = labelDs.map(lambda g,a: {"gender": g, "age": a}, num_parallel_calls=tf.data.AUTOTUNE, name='label_dict')
    
    # Combine Images and Labels into dataset
    ds = tf.data.Dataset.zip((imageDs, labelDs))
    
    # Configure Performance
    if(configureDs is not None):
        ds = configureDs(ds)
    
    # Pre Batch Preprocessing
    for n,f in preBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    # Batch
    if(batchSize is not None):
        ds = ds.batch(batchSize, name="ds_batch")
    
    # Post Batch Preprocessing
    for n,f in postBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    return ds

### Preprocessings

In [13]:
def trainConfigPerformance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.repeat()
    return ds

In [14]:
def valConfigPerformance(ds):
    #ds = ds.cache()
    return ds

#### Preprocessing steps according to the reference paper

In [15]:
def trainPreprocessA(img, lab):
        
        # img = tf.image.random_crop(img, [CROP_HEIGHT, CROP_WIDTH, 3])
        
        ud = tf.random.uniform([], dtype=tf.float32)
        udCond = tf.less(ud, 0.5)
        
        img = tf.cond(udCond, lambda: tf.image.flip_up_down(img), lambda: img)
        
        lr = tf.random.uniform([], dtype=tf.float32)
        lrCond = tf.less(lr, 0.5)
        
        img = tf.cond(lrCond, lambda: tf.image.flip_left_right(img), lambda: img)
        
        img = tf.image.random_brightness(img, 63/255)
        
        img = tf.image.random_contrast(img, 0.2, 1.8) 
        
        return img, lab

In [16]:
def valPreprocessA(img, lab):
        
        img = tf.image.crop_to_bounding_box(img,
                                            (IMG_HEIGHT-CROP_HEIGHT) // 2,
                                            (IMG_WIDTH-CROP_WIDTH) // 2,
                                            CROP_HEIGHT,
                                            CROP_WIDTH)
        
        return img, lab

In [17]:
def imageStandardize(img, lab):
    
    img = tf.image.per_image_standardization(img)
    
    return img, lab

#### Label extractor 

In [18]:
def extractGenderLabel(img, lab):
    
    lab = lab['gender']
    
    return img, lab

In [19]:
def extractAgeLabel(img, lab):
    
    lab = lab['age']
    
    return img, lab

### Generate Folds for K-Folds validation

In [20]:
def generateFoldDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, valBatchSize, trainConfigureDs=None, trainPreBatch=[], trainPostBatch=[], valConfigureDs=None, valPreBatch=[], valPostBatch=[]):
    '''
        Returns list of (train, validation) datasets
    '''
    
    N = len(foldFiles)
    
    folds = []
    
    for i in range(N):
        
        trainFiles = foldFiles[:i]
        if(i < N-1):
            trainFiles.extend(foldFiles[i+1:])
            
        valFiles = foldFiles[i]
        
        trainDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, configureDs=trainConfigureDs, preBatch=trainPreBatch, postBatch=trainPostBatch)
        valDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, valBatchSize, configureDs=valConfigureDs, preBatch=valPreBatch, postBatch=valPostBatch)
        
        folds.append((trainDs, valDs))
        
    return folds

In [21]:
trainPreBatch = [("train_process_a", trainPreprocessA), 
                 ("train_standardize", imageStandardize), 
                 ("train_extract_gender", extractGenderLabel)]

valPreBatch = [("val_standardize", imageStandardize), 
               ("val_extract_gender", extractGenderLabel)]

In [22]:
folds = generateFoldDs(foldFiles, "adience/faces", "coarse_tilt_aligned_face", genderMap, ageMap, 2, 8, 
                       IMG_WIDTH, IMG_HEIGHT, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, 
                       trainConfigureDs=trainConfigPerformance, trainPreBatch=trainPreBatch, 
                       valConfigureDs=valConfigPerformance, valPreBatch=valPreBatch)

2022-11-09 21:25:24.770151: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 21:25:24.770868: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 21:25:24.771149: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 21:25:24.771361: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [23]:
len(folds)

5

### Check data

In [24]:
for x,y in folds[0][0].take(1):
    pass

In [25]:
x

<tf.Tensor: shape=(4, 256, 256, 3), dtype=float32, numpy=
array([[[[-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         ...,
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ]],

        [[-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         ...,
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ]],

        [[-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         ...,
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ],
         [-1.2995634, -1.2804654, -1.251098 ]],

        ...,

        [[-1.2995634, -1.2804654, 

In [26]:
y

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)>

### Model

In [27]:
def createModel():
    inp = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    conv1 = tf.keras.layers.Conv2D(96, 7, strides=(4,4), activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv1')
    pool1 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool1')

    conv2 = tf.keras.layers.Conv2D(256, 5, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv2')
    pool2 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool2')

    conv3 = tf.keras.layers.Conv2D(384, 3, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv3')
    pool3 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool3')

    flat1 = tf.keras.layers.Flatten(name='flat1')
    fc1 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc1')
    do1 = tf.keras.layers.Dropout(0.5, name='do1')

    fc2 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc2')
    do2 = tf.keras.layers.Dropout(0.5, name='do2')

    fc3 = tf.keras.layers.Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='fc3')

    o = conv1(inp)
    o = pool1(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [2, 2], [2, 2], [0, 0]])
    o = conv2(o)
    o = pool2(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [1, 1], [1, 1], [0, 0]])
    o = conv3(o)
    o = pool3(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = flat1(o)
    o = fc1(o)
    o = do1(o)


    o = fc2(o)
    o = do2(o)

    o = fc3(o)

    model = tf.keras.models.Model(inputs=inp, outputs=o, name='cnn_7')
    
    return model

### Compile Model

In [28]:
STEPS_PER_EPOCH = 1000

EPOCH = 300

START_EPOCH = 0
END_EPOCH = START_EPOCH + EPOCH

#### Learning Rate Schedule

In [29]:
def schedule(epoch, lr):
    if(epoch == 10000*50//STEPS_PER_EPOCH):
        return lr * 0.1
    else:
        return lr

In [30]:
logBasePath = "log/CNN_7"
logPrefix = "log"

In [31]:
for i, (train, val) in enumerate([folds[0]]):
    
    model = createModel()
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.001)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    
    tboardCb = tf.keras.callbacks.TensorBoard(log_dir=f"{logBasePath}/{logPrefix}_{i}", histogram_freq=1, profile_batch=(2,10))
    lrSchedule = tf.keras.callbacks.LearningRateScheduler(schedule)
    
    cbs = [tboardCb]
    
    model.evaluate(val)
    model.fit(train, initial_epoch=START_EPOCH, epochs=END_EPOCH, steps_per_epoch=STEPS_PER_EPOCH, callbacks=cbs, validation_data=val)

2022-11-09 21:25:27.477267: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-09 21:25:27.477286: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-09 21:25:27.477306: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-11-09 21:25:27.562593: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-11-09 21:25:27.563805: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2022-11-09 21:25:28.098632: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  55/3528 [..............................] - ETA: 9s - loss: 0.6787 - accuracy: 0.6136   

2022-11-09 21:25:28.861433: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3528/3528 [==============================] - 10s 2ms/step - loss: 0.6996 - accuracy: 0.4768
Epoch 1/300
   1/1000 [..............................] - ETA: 7:19 - loss: 0.8364 - accuracy: 0.2500

2022-11-09 21:25:38.061933: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-09 21:25:38.061953: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.


...

1000/1000 [==============================] - 14s 14ms/step - loss: 0.6849 - accuracy: 0.5598 - val_loss: 0.6961 - val_accuracy: 0.5232
Epoch 61/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6940 - accuracy: 0.4967 - val_loss: 0.6940 - val_accuracy: 0.4768
Epoch 62/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6858 - accuracy: 0.5533 - val_loss: 0.6930 - val_accuracy: 0.5232
Epoch 63/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6745 - accuracy: 0.5698 - val_loss: 0.6963 - val_accuracy: 0.4768
Epoch 64/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6878 - accuracy: 0.5405 - val_loss: 0.6923 - val_accuracy: 0.5232
Epoch 65/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6932 - accuracy: 0.5150 - val_loss: 0.6920 - val_accuracy: 0.5232
Epoch 66/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6703 - accuracy: 0.5930 - val_loss: 0.6926 

1000/1000 [==============================] - 14s 14ms/step - loss: 0.6896 - accuracy: 0.5418 - val_loss: 0.6920 - val_accuracy: 0.5232
Epoch 116/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6803 - accuracy: 0.5518 - val_loss: 0.6921 - val_accuracy: 0.5232
Epoch 117/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6898 - accuracy: 0.5278 - val_loss: 0.6949 - val_accuracy: 0.4768
Epoch 118/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6929 - accuracy: 0.4995 - val_loss: 0.6972 - val_accuracy: 0.4768
Epoch 119/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6767 - accuracy: 0.5838 - val_loss: 0.6934 - val_accuracy: 0.5232
Epoch 120/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6898 - accuracy: 0.5245 - val_loss: 0.6924 - val_accuracy: 0.5232
Epoch 121/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6942 - accuracy: 0.5070 - val_loss: 0

1000/1000 [==============================] - 14s 14ms/step - loss: 0.5172 - accuracy: 0.7477 - val_loss: 0.5507 - val_accuracy: 0.7155
Epoch 171/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5140 - accuracy: 0.7465 - val_loss: 0.6219 - val_accuracy: 0.6503
Epoch 172/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5118 - accuracy: 0.7322 - val_loss: 0.5711 - val_accuracy: 0.6989
Epoch 173/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5211 - accuracy: 0.7405 - val_loss: 0.5679 - val_accuracy: 0.7090
Epoch 174/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5022 - accuracy: 0.7563 - val_loss: 0.5307 - val_accuracy: 0.7307
Epoch 175/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5140 - accuracy: 0.7470 - val_loss: 0.5322 - val_accuracy: 0.7306
Epoch 176/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5121 - accuracy: 0.7312 - val_loss: 0

1000/1000 [==============================] - 14s 14ms/step - loss: 0.3826 - accuracy: 0.8242 - val_loss: 0.3854 - val_accuracy: 0.8236
Epoch 226/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3289 - accuracy: 0.8587 - val_loss: 0.4147 - val_accuracy: 0.8148
Epoch 227/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3223 - accuracy: 0.8572 - val_loss: 0.4491 - val_accuracy: 0.7948
Epoch 228/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3456 - accuracy: 0.8395 - val_loss: 0.4352 - val_accuracy: 0.7900
Epoch 229/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3818 - accuracy: 0.8225 - val_loss: 0.3747 - val_accuracy: 0.8282
Epoch 230/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2925 - accuracy: 0.8783 - val_loss: 0.4068 - val_accuracy: 0.8187
Epoch 231/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3104 - accuracy: 0.8668 - val_loss: 0

1000/1000 [==============================] - 14s 14ms/step - loss: 0.2217 - accuracy: 0.9072 - val_loss: 0.2579 - val_accuracy: 0.8884
Epoch 281/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2330 - accuracy: 0.8990 - val_loss: 0.2430 - val_accuracy: 0.8950
Epoch 282/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2663 - accuracy: 0.8802 - val_loss: 0.3696 - val_accuracy: 0.8509
Epoch 283/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2001 - accuracy: 0.9190 - val_loss: 0.2563 - val_accuracy: 0.8913
Epoch 284/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2141 - accuracy: 0.9120 - val_loss: 0.2611 - val_accuracy: 0.8870
Epoch 285/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2610 - accuracy: 0.8885 - val_loss: 0.2521 - val_accuracy: 0.8890
Epoch 286/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2075 - accuracy: 0.9133 - val_loss: 0